This notebook prepares some files to upload data to SRA

More documentation:

https://www.ncbi.nlm.nih.gov/sra/docs/submitportal/

https://www.ncbi.nlm.nih.gov/sra/docs/submitfiles/

In [2]:
import pandas as pd
import numpy as np
import os

First, let's grab all the samples we'll need to upload data for.

In [4]:
fsamples = '../../final/supp_files/patients_with_sites_sampled.csv'
df = pd.read_csv(fsamples, index_col=0)
df.head()

,bal,gastric_fluid,throat_swab,stool
04-080-7,True,True,True,False
01-299-7,False,False,True,False
04-167-8,False,True,False,False
04-087-1,True,True,True,False
042-6-F1,False,False,True,False


In [12]:
allsubjects = df.index.tolist()

In [16]:
dirsamples = '../../data/patients/'
files = os.listdir(dirsamples)

files = [f for f in files if f.endswith('.samples.txt')]
files = [dirsamples + f for f in files]

#dirsamples = '../../final/patients/'
#files = os.listdir(dirsamples)
#files = [f for f in files if 'samples' in f]
#files

In [18]:
# Read in all the samples 
allsamples = []
for fname in files:
#    path = os.path.join('../../final/patients/', fname)
    path = fname
    with open(path, 'r') as fh:
        lines = fh.readlines()
        #print(fname, len(lines))
        samples = [l.strip() for l in lines]
        allsamples += samples

In [19]:
## How many total/unique samples are in these figures?
len(allsamples), len(set(allsamples))

(4378, 450)

In [20]:
# Get samples and connect metadata to these subjects
fmeta = '../../data/clean/rosen.metadata.clean'
meta = pd.read_csv(fmeta, sep='\t', index_col=0)

In [21]:
setsamples = list(set(allsamples))
meta.loc[setsamples].groupby('site').size()

site
bal              102
gastric_fluid    147
stool             25
throat_swab      176
dtype: int64

In [7]:
# Hm, I was expecting 20 stool samples. Let's look into this more
# Punchline: 20 stool samples have paired throat sample, but I have 25 total samples in the PCoA plot
stoolsamples = meta.loc[setsamples].query('site == "stool"').index.tolist()

In [8]:
# stool = []
# for fname in files:
#     path = os.path.join('../../data/patients/', fname)
#     with open(path, 'r') as fh:
#         samples = [i.strip() for i in fh.readlines()]
#         r = [i for i in samples if i in stoolsamples]
#         if len(r) > 0:
#             print(fname, r)
#             stool.append(r)

('figure3.site_classifiers.samples.txt', ['03-181-7SI', '03-125-1SI', '03-055-1SI', '03-076-8SI', '03-149-1SI', '03-226-4SI', '03-115-7SI', '03-114-2SI', '03-116-1SI', '03-146-6SI', '03-199-7SI', '03-156-7SI', '13-058-2SI', '03-069-2SI', '13-089-1SI', '03-138-9SI', '03-153-7SI', '03-085-9SI', '03-088-2SI', '03-105-8SI'])
('figure2.pcoa.samples.txt', ['03-055-1SI', '03-069-2SI', '03-085-9SI', '03-088-2SI', '03-105-8SI', '03-114-2SI', '03-115-7SI', '03-116-1SI', '03-125-1SI', '13-058-2SI', '13-089-1SI', '01-165-8SI', '03-076-8SI', '03-138-9SI', '03-146-6SI', '03-149-1SI', '03-150-8SI', '03-153-7SI', '03-156-7SI', '03-178-6SI', '03-181-7SI', '03-199-7SI', '03-225-1SI', '03-226-4SI', '03-272-3SI'])
('figure3.within_patient_beta_div.samples.txt', ['03-181-7SI', '03-125-1SI', '03-149-1SI', '03-055-1SI', '03-076-8SI', '03-226-4SI', '03-115-7SI', '03-114-2SI', '03-116-1SI', '03-199-7SI', '03-156-7SI', '13-058-2SI', '03-146-6SI', '03-069-2SI', '13-089-1SI', '03-138-9SI', '03-153-7SI', '03-085-9

In [9]:
# print([i for i in stool[0] if i not in stool[1]])
# print([i for i in stool[0] if i not in stool[2]])

# print([i for i in stool[1] if i not in stool[0]])
# print([i for i in stool[1] if i not in stool[2]])

# print([i for i in stool[2] if i not in stool[0]])
# print([i for i in stool[2] if i not in stool[1]])


[]
[]
['01-165-8SI', '03-150-8SI', '03-178-6SI', '03-225-1SI', '03-272-3SI']
['01-165-8SI', '03-150-8SI', '03-178-6SI', '03-225-1SI', '03-272-3SI']
[]
[]


There are 5 stool samples that are in the PCoA plot that are not in the classifier or within-patinet beta diversity
This makes sense: I have 25 stool samples, but only 20 with a paired throat sample.

# Prepare data for SRA

Now that I've gotten all the samples I use in all the figures, I need to prepare a tab-delimited file for upload to the SRA.

A template can be found: https://www.ncbi.nlm.nih.gov/biosample/docs/templates/packages/MIMARKS.survey.human-associated.4.0.xlsx

In [10]:
fields = ['sample_name', 'organism', 'collection_date', 
          'env_biome', 'env_feature', 'env_material',
          'geo_loc_name', 'host', 'lat_lon', 
          'subject_id']
common_fields = ['organism', 'env_biome', 'geo_loc_name', 'host', 'lat_lon', 'collection_date']

Some notes on these mandatory fields:

- `organism` is "the most descriptive name for this sample (to the species, if relevant)." These are mixed communities, and it looks like usually people put "human gut metagenome" so here I'll just put "human-associated metagenome"    
- `collection_date`: I don't have this, but I'll put sequencing_date as a differnet column with batch year       
- all the env fields: from reading a bit more about them on the [attributes documentation](https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/) and looking at a few examples, here's what I think:   
    - `env_biome` should be human body or human-associated habitat   
    - `env_feature` is a broad descriptor of the environment, so "lung", "oropharynx", "stomach"    
    - `env_material` is what the samples actually were: "BAL, oropharyngeal_swab, and gastric fluid   
- `geo_loc_name` will be USA: Boston   
- `host`: Homo sapiens   
- `lat_lon`: omg why is this a required attribute. I'll put "missing" here.

In [11]:
# Set up dataframe
sra = pd.DataFrame(columns=common_fields)
sra['sample_name'] = setsamples
print(sra.shape)

# Populate the common attributes
sra['organism'] = 'human-associated metagenome'
sra['env_biome'] = 'human-associated habitat'
sra['geo_loc_name'] = 'USA: Boston'
sra['host'] = "Homo sapiens"
sra['lat_lon'] = 'missing'
sra['collection_date'] = 'missing'
print(sra.shape)

(450, 7)
(450, 7)


In [12]:
metacols = ['site', 'subject_id', 'batch']
sra = pd.merge(sra, meta[metacols], how='left', 
               left_on='sample_name', right_index=True)
sra.head()

,organism,env_biome,geo_loc_name,host,lat_lon,collection_date,sample_name,site,subject_id,batch
0,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,04-074-1T,throat_swab,04-074-1,2014
1,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,02-164-1G,gastric_fluid,02-164-1,2014
2,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,04-262-5T,throat_swab,04-262-5,2016
3,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,04-074-1G,gastric_fluid,04-074-1,2014
4,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,04-074-1B,bal,04-074-1,2014


In [13]:
feat_map = {'throat_swab': 'oropharynx', 'bal': 'lung', 'gastric_fluid': 'stomach', 'stool': 'gut'}
sra['env_feature'] = sra['site'].map(lambda x: feat_map[x])

mat_map = {'throat_swab': 'oropharyngeal_swab', 'bal': 'bronchoalveolar_lavage', 
           'gastric_fluid': 'gastric_fluid', 'stool': 'feces'}
sra['env_material'] = sra['site'].map(lambda x: mat_map[x])

sra = sra.rename(columns={'batch': 'sequencing_date'})
sra = sra.drop('site', axis=1)

sra.head()

,organism,env_biome,geo_loc_name,host,lat_lon,collection_date,sample_name,subject_id,sequencing_date,env_feature,env_material
0,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,04-074-1T,04-074-1,2014,oropharynx,oropharyngeal_swab
1,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,02-164-1G,02-164-1,2014,stomach,gastric_fluid
2,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,04-262-5T,04-262-5,2016,oropharynx,oropharyngeal_swab
3,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,04-074-1G,04-074-1,2014,stomach,gastric_fluid
4,human-associated metagenome,human-associated habitat,USA: Boston,Homo sapiens,missing,missing,04-074-1B,04-074-1,2014,lung,bronchoalveolar_lavage


In [14]:
[i for i in fields if i not in sra.columns]

[]

In [15]:
sra.to_csv('../../final/supp_files/biosample_attributes.SUB3758953.txt', sep='\t', index=False)

# Download data on AWS

First, I need to get the name of samples in the two batches (they're in different folders on AWS).


In [16]:
# 2014 data
with open('sample_list.2014.txt', 'w') as f:
    f.write('\n'.join(sra.query('sequencing_date == 2014')['sample_name'].tolist()) + '\n')

In [17]:
# 2016 data is in format subjectID-sampleID_R1.fastq, except for the following samples:
# ['58-4-TI', '50-4-TI', '48-1-TI', '57-9-TI', '51-6-TI', '55-1-TI']
# e.g. 04-200-1-G_R1.fastq
samples2016 = sra.query('sequencing_date == 2016')['sample_name'].tolist()
normal_files = ['58-4-TI', '50-4-TI', '48-1-TI', '57-9-TI', '51-6-TI', '55-1-TI']
files2016 = []
for s in samples2016:
    if s in normal_files:
        files2016.append(s + '_R1.fastq')
        files2016.append(s + '_R2.fastq')
    else:
        news = s.rsplit('-', 1)[0] + '-' + s.rsplit('-', 1)[1][0] + '-' + s.rsplit('-', 1)[1][1:]
        files2016.append(news + '_R1.fastq')
        files2016.append(news + '_R2.fastq')
with open('file_list.2016.txt', 'w') as f:
    f.write('\n'.join(files2016) + '\n')

Upload these files to the almlab node.

#### Download each file (fwd and rev) to almlab node 

2014 samples: just read sample IDs from file:

```
while read f; do aws s3 cp s3://almlab.bucket/duvallet/rosen_data/2014_data/fastq_R1_R2/${f}_R1.fastq .; aws s3 cp s3://almlab.bucket/duvallet/rosen_data/2014_data/fastq_R1_R2/${f}_R2.fastq .; done < ../sample_list.2014.txt 
```

2016 samples: read in individual files:

```
while read f; do aws s3 cp s3://almlab.bucket/duvallet/rosen_data/2016_data/fastq_split/${f} .; done < ../file_list.2016.txt
```

#### Check that each sample ID in the sra_metadata.SUB3758953.txt file has a fwd and rev file    

2016 files are good, 2014 file has sample "04-087-1G" which has lowercase "g" in the file name.

Just do this one manually:

```
aws s3 cp s3://almlab.bucket/duvallet/rosen_data/2014_data/fastq_R1_R2/04-087-1g_R1.fastq .
aws s3 cp s3://almlab.bucket/duvallet/rosen_data/2014_data/fastq_R1_R2/04-087-1g_R2.fastq .
```

There are 450 samples, so should be 900 total files. Check!

#### Write the file names in the SRA Metadata file   


#### ftp the files to SRA ([doc](https://www.ncbi.nlm.nih.gov/sra/docs/submitfiles/))

Still TODO:

- fill in the sequencer and platform info after I get methods from Scott

# SRA Metadata

https://www.ncbi.nlm.nih.gov/sra/docs/submitportal/#6-sra-metadata

This is different, and has info about the way each sample was processed etc

strategy will be AMPLICON   
source will be METAGENOMIC   
selection will be PCR   
platform is TBD   

Other columns

bioproject_accession - leave empty   
sample_name - "If you created samples in the SRA wizard, provide names of samples that you just created in the column sample_name"   
library_ID - just needs to be unique per sample, so probably just make it sample name?   
title - Aerodigestive microbiome: 16S sequencing of oropharyngeal, gastric, and BAL samples

library_strategy - AMPLICON     
library_source - METAGENOMIC     
library_selection - PCR    
library_layout - PAIRED    
platform - TBD    
instrument_model - TBD    
design_description - TBD ("Free-form description of the methods used to create the sequencing library; a brief 'materials and methods' section."    
filetype - fastq    
filename - forward read (?)    
filename2 - reverse read (?)  


In [18]:
cols = ['bioproject_accession', 'sample_name', 'library_ID', 
        'title', 'library_strategy', 'library_source', 
        'library_selection', 'library_layout', 'platform', 
        'instrument_model', 'design_description', 
        'filetype', 'filename', 'filename2']

# Set up SRA metadata
srameta = pd.DataFrame(columns=cols)
srameta['sample_name'] = setsamples
srameta['library_ID'] = setsamples

# Common items
srameta['library_strategy'] = 'AMPLICON'
srameta['library_source'] = 'METAGENOMIC'
srameta['library_selection'] = 'PCR'
srameta['library_layout'] = 'PAIRED'
srameta['filetype'] = 'fastq'

srameta['title'] = 'Aerodigestive microbiome: 16S sequencing of oropharyngeal, gastric, and BAL samples'
srameta['platform'] = 'ILLUMINA'
srameta['instrument_model'] = 'Illumina MiSeq'
srameta['design_description'] = ('Samples suspended in Tris-Saline buffer were centrifuged '
                                 'for 3 minutes at 10,000 rcf. DNA was extracted from the sample '
                                 'pellet with the Qiagen DNeasy PowerSoil Kit, with the following '
                                 'modifications: protein precipitation in one step using 100 μL of '
                                 'each C2 and C3 solutions, and column centrifugation at 10,000 rcf '
                                 'for 10 minutes.')

srameta.head()

,bioproject_accession,sample_name,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2
0,NaN,04-074-1T,04-074-1T,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,NaN,NaN
1,NaN,02-164-1G,02-164-1G,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,NaN,NaN
2,NaN,04-262-5T,04-262-5T,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,NaN,NaN
3,NaN,04-074-1G,04-074-1G,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,NaN,NaN
4,NaN,04-074-1B,04-074-1B,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,NaN,NaN


In [19]:
# File names
s2file = {}
s2file_r2 = {}

# 2014 files
samples2014 = sra.query('sequencing_date == 2014')['sample_name'].tolist()

s2file = {s: s + '_R1.fastq' for s in samples2014}
s2file["04-087-1G"] = "04-087-1g_R1.fastq"

s2file_r2 = {s: s + '_R2.fastq' for s in samples2014}
s2file_r2["04-087-1G"] = "04-087-1g_R2.fastq"


# 2016 files
for s in samples2016:
    if s in normal_files:
        s2file[s] = s + '_R1.fastq'
        s2file_r2[s] = s + '_R2.fastq'
    else:
        news = s.rsplit('-', 1)[0] + '-' + s.rsplit('-', 1)[1][0] + '-' + s.rsplit('-', 1)[1][1:]
        s2file[s] = news + '_R1.fastq'
        s2file_r2[s] = news + '_R2.fastq'

In [20]:
srameta['filename'] = srameta['sample_name'].apply(lambda x: s2file[x])
srameta['filename2'] = srameta['sample_name'].apply(lambda x: s2file_r2[x])

In [21]:
srameta.to_csv('../../final/supp_files/sra_metadata.SUB3758953.txt', sep='\t', index=False)
srameta.head()

,bioproject_accession,sample_name,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2
0,NaN,04-074-1T,04-074-1T,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,04-074-1T_R1.fastq,04-074-1T_R2.fastq
1,NaN,02-164-1G,02-164-1G,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,02-164-1G_R1.fastq,02-164-1G_R2.fastq
2,NaN,04-262-5T,04-262-5T,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,04-262-5-T_R1.fastq,04-262-5-T_R2.fastq
3,NaN,04-074-1G,04-074-1G,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,04-074-1G_R1.fastq,04-074-1G_R2.fastq
4,NaN,04-074-1B,04-074-1B,Aerodigestive microbiome: 16S sequencing of or...,AMPLICON,METAGENOMIC,PCR,PAIRED,ILLUMINA,Illumina MiSeq,Samples suspended in Tris-Saline buffer were c...,fastq,04-074-1B_R1.fastq,04-074-1B_R2.fastq


Then submit at https://submit.ncbi.nlm.nih.gov/subs/sra/SUB3758953/metadata